In [1]:
import polars as pl

import nwec.utility_reporting.arrearages
import nwec.utils.excel
from nwec.constants import DATA

YEAR = 2024
NUM_MONTHS = 3
COLS_PER_MONTH = 8
KLI_COLS_PER_MONTH = 4
SHEET_SEARCH_STRING = "Past Due Balances"
ARREARAGE_SEARCH_STRING = "Past-due balances by customer class"
KLI_SEARCH_STRING = "known low-income"
spreadsheet = DATA / f"utility_reporting/nwng/nwng_{YEAR}.xlsx"

In [2]:
sheet_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, SHEET_SEARCH_STRING)
df = pl.read_excel(spreadsheet, sheet_id=sheet_index, has_header=False)
arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(
    df, NUM_MONTHS, COLS_PER_MONTH, ARREARAGE_SEARCH_STRING
)
kli_arrearages = nwec.utility_reporting.arrearages.get_arrearages_df(
    df, NUM_MONTHS, KLI_COLS_PER_MONTH, KLI_SEARCH_STRING
)

# Arrearages

In [3]:
date_to_zip_offset = 1
source_date_format = "%b %Y"

arrearages = nwec.utility_reporting.arrearages.rename_arrearage_columns(
    arrearages, COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [4]:
arrearages.sum()

Zip Code,April 2024 30 Days Amt,April 2024 30 Day Count,April 2024 60 Days Amt,April 2024 60 Days Count,April 2024 90 Days + Amt,April 2024 90 Days + Count,April 2024 Total Arrearages,April 2024 Total Count,May 2024 30 Days Amt,May 2024 30 Day Count,May 2024 60 Days Amt,May 2024 60 Days Count,May 2024 90 Days + Amt,May 2024 90 Days + Count,May 2024 Total Arrearages,May 2024 Total Count,June 2024 30 Days Amt,June 2024 30 Day Count,June 2024 60 Days Amt,June 2024 60 Days Count,June 2024 90 Days + Amt,June 2024 90 Days + Count,June 2024 Total Arrearages,June 2024 Total Count
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,1.3008e6,7338.0,530589.64,4427.0,262109.24,2390.0,2.0935e6,14155.0,958601.13,6157.0,538972.22,4543.0,315149.72,2963.0,1.8127e6,13663.0,755043.46,5485.0,469782.58,3917.0,390257.18,3976.0,1.6151e6,13378.0


# KLI Arrearages

In [5]:
date_to_zip_offset = 1
source_date_format = "%b %Y"

kli_arrearages = nwec.utility_reporting.arrearages.rename_arrearage_columns(
    kli_arrearages, KLI_COLS_PER_MONTH, date_to_zip_offset, source_date_format
)

In [6]:
kli_arrearages.sum()

Zip Code,April 2024 30 Days,April 2024 60 Days,April 2024 90 Days +,April 2024 Total Arrearages,May 2024 30 Days,May 2024 60 Days,May 2024 90 Days +,May 2024 Total Arrearages,June 2024 30 Days,June 2024 60 Days,June 2024 90 Days +,June 2024 Total Arrearages
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,16732.52,14732.93,21041.81,52507.26,5867.38,5554.94,10261.1,21683.42,2729.54,2837.67,5641.03,11208.24
